# Simple tool formatting with HF
Showcasing how to use the new template format from HF for tool calling

## Loading the models

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model_id = "/home/ubuntu/projects/llama-recipes/models--llhf--Meta-Llama-3.1-8B-Instruct/snapshots/9fd0b760200bab0a7af5e24c14f1283ecdb4765f"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )




Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Setting up the system messages and default tools 

In [29]:
dialogs = [
    [
        {"role": "system", "content": "You are a helpful chatbot"},
        {"role": "user", "content": "What is the weather today in San Francisco?"},
    ],
    [
        {"role": "system", "content": "You are a helpful chatbot"},
        {"role": "user", "content": "What is the weather today in San Francisco?"},
    ],
  ]

messages = [
        {"role": "system", "content": "You are a helpful chatbot"},
        {"role": "user", "content": "What is the weather today in San Francisco?"},
    ]

builtin_tools = ["code_interpreter", "wolfram_alpha", "brave_search"]

json_tools = [ 
    { "type": "function",
      "function": {
          "name": "spotify_trending_songs",
          "description": "Get top trending songs on Spotify",
          "parameters": {
            "n": {
              "param_type": "int",
              "description": "Number of trending songs to get",
              "required": "true"
            }
          }
        }
   },
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_rain_probability",
        "description": "Get the probability of rain for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            }
          },
          "required": ["location"]
        }
      }
    }
]


json_tools = [ 
    { 
      "tool_name": "spotify_trending_songs",
      "description": "Get top trending songs on Spotify",
      "parameters": {
        "n": {
          "param_type": "int",
          "description": "Number of trending songs to get",
          "required": "true"
        }
      }
    
   },
    {
      "type": "function",
      "function": {
        "name": "get_rain_probability",
        "description": "Get the probability of rain for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            }
          },
          "required": ["location"]
        }
      }
    }
]



## Converting to input ids and checking how the prompt format was applied

In [21]:
# for messages in dialog:
# Shouldn't output the Environment instruction, but it is.
input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

print(tokenizer.decode(input_ids[0], skip_special_tokens=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You are a helpful chatbot<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the weather today in San Francisco?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [22]:
input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        builtin_tools=builtin_tools
    ).to(model.device)

print(tokenizer.decode(input_ids[0], skip_special_tokens=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: wolfram_alpha, brave_search

Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You are a helpful chatbot<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the weather today in San Francisco?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [30]:
input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
            custom_tools=json_tools,
        ).to(model.device)
    
print(tokenizer.decode(input_ids[0], skip_special_tokens=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You are a helpful chatbot<|eot_id|><|start_header_id|>user<|end_header_id|>

Use the function'spotify_trending_songs' to 'Get top trending songs on Spotify':
{"name": "spotify_trending_songs", "description": "Get top trending songs on Spotify", "parameters": {
    "n": {
        "param_type": "int",
        "description": "Number of trending songs to get",
        "required": "true"
    }
}Use the function 'get_rain_probability' to 'Get the probability of rain for a specific location':
{"name": "get_rain_probability", "description": "Get the probability of rain for a specific location", "parameters": {
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g., San Francisco, CA"
        }
    },
    "required": [
        "location"
    ]
}

Think very care

In [ ]:
input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
            custom_tools=json_tools,
            builtin_tools=builtin_tools
        ).to(model.device)
    
print(tokenizer.decode(input_ids[0], skip_special_tokens=False))

In [14]:
print(input_ids[0])
print(tokenizer.decode(input_ids[0], skip_special_tokens=False))

tensor([128000, 128006,   9125, 128007,    271,  13013,     25,   6125,  27993,
           198,  38766,   1303,  33025,   2696,     25,   6790,    220,   2366,
            18,    198,  15724,   2696,     25,    220,   1419,  10263,    220,
          2366,     19,    271,   2675,    527,    264,  11190,   6369,   6465,
        128009, 128006,    882, 128007,    271,   3923,    374,    279,   9282,
          3432,    304,   5960,  13175,     30, 128009, 128006,  78191, 128007,
           271])
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You are a helpful chatbot<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the weather today in San Francisco?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




## Running inference 


In [15]:
attention_mask = torch.ones_like(input_ids)
outputs = model.generate(
    input_ids,
    max_new_tokens=400,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    attention_mask=attention_mask,
)
response = outputs[0][input_ids.shape[-1]:]
print("\nOutput:\n")
print(tokenizer.decode(response, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1850: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



Output:

I'm just an AI, I don't have access to real-time weather information. However, I can suggest some ways for you to find out the current weather in San Francisco.

You can check the weather forecast on websites such as:

1. AccuWeather (accuweather.com)
2. Weather.com (weather.com)
3. National Weather Service (weather.gov)

You can also check the weather on your smartphone by downloading a weather app such as Dark Sky or Weather Underground.

If you want to know the current weather in San Francisco, I can suggest some general information about the city's climate. San Francisco has a mild climate year-round, with cool summers and mild winters. The city is known for its foggy weather, especially during the summer months. The average high temperature in San Francisco is around 67°F (19°C), while the average low temperature is around 54°F (12°C).
